In [1]:
import pandas as pd
import torch
from preprocessing import *
import time
from PC_transformer import PCTransformer

set_seed(43)

In [2]:
df = pd.read_csv(r'output_csv\ANL-Intrepid-2009-1.swf.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68936 entries, 0 to 68935
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   job_id                    68936 non-null  float64
 1   submit_time               68936 non-null  float64
 2   wait_time                 68936 non-null  float64
 3   run_time                  68936 non-null  float64
 4   num_allocated_processors  68936 non-null  float64
 5   avg_cpu_time_used         68936 non-null  float64
 6   used_memory               68936 non-null  float64
 7   requested_processors      68936 non-null  float64
 8   requested_time            68936 non-null  float64
 9   requested_memory          68936 non-null  float64
 10  status                    68936 non-null  float64
 11  user_id                   68936 non-null  float64
 12  group_id                  68936 non-null  float64
 13  executable_id             68936 non-null  float64
 14  queue_

In [3]:
df.head()

,job_id,submit_time,wait_time,run_time,num_allocated_processors,avg_cpu_time_used,used_memory,requested_processors,requested_time,requested_memory,status,user_id,group_id,executable_id,queue_id,partition_id,preceding_job_id,think_time
0,1.0,0.0,6680.0,7560.0,2048.0,-1.0,-1.0,2048.0,10800.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0
1,2.0,7.0,14297.0,7568.0,2048.0,-1.0,-1.0,2048.0,10800.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0
2,3.0,1590.0,17322.0,7561.0,2048.0,-1.0,-1.0,2048.0,10800.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0
3,4.0,2205.0,61.0,14972.0,8192.0,-1.0,-1.0,8192.0,21600.0,-1.0,-1.0,2.0,-1.0,-1.0,2.0,-1.0,-1.0,-1.0
4,5.0,2566.0,17357.0,7571.0,2048.0,-1.0,-1.0,2048.0,10800.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0


In [4]:
feature_columns = ['requested_processors', 'requested_time', 'submit_time', 'wait_time', 'user_id', 'queue_id']
target_column = 'run_time'

# Hyperparameters
input_dim = len(feature_columns)    # Example input dimension
d_model = 64      # Model dimension
num_heads = 8     # Number of attention heads
d_ff = 256        # Feed-forward dimension
num_layers = 3    # Number of transformer blocks
output_dim = 1    # Output dimension
batch_size = 128  # Batch size
seq_len = 20      # Length of data group
num_epochs = 20   # Number of training epochs
dropout = 0.2    # Dropout rate
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

dataloaders, scaler = prepare_data_seq(df, feature_columns, target_column, statuss=-1, seq_len=seq_len, batch_size=batch_size)

In [5]:
start = time.time()

# Create model
model = PCTransformer(input_dim, d_model, num_heads, d_ff, num_layers, output_dim, dropout).to(device)

# Train the model
model.train_model(model, dataloaders, num_epochs, device, scaler)

end = time.time()

# Final evaluation on test set
test_rmse, test_mae, test_mse, test_r2 = model.evaluate_model(model, dataloaders['test'], device, scaler, input_dim)
print(f'\nFinal Test Metrics:')
print(f'Test RMSE: {test_rmse:.4f}, Test MAE: {test_mae:.4f}, Test MSE: {test_mse:.4f}, Test R2: {test_r2:.4f}')
print(f'Training time: {end - start:.2f} seconds')

Epoch [1/20]
Train Loss: 0.2463, Val Loss: 0.2767
Val RMSE: 9641.0032, Val MAE: 5273.7607, Val MSE: 92948942.0792, Val R2: 0.1353
Epoch [2/20]
Train Loss: 0.2008, Val Loss: 0.2179
Val RMSE: 8137.6004, Val MAE: 4445.1273, Val MSE: 66220540.6695, Val R2: 0.3839
Epoch [3/20]
Train Loss: 0.1189, Val Loss: 0.1506
Val RMSE: 6729.3324, Val MAE: 3335.6424, Val MSE: 45283914.3649, Val R2: 0.5787
Epoch [4/20]
Train Loss: 0.1042, Val Loss: 0.1524
Val RMSE: 6729.9105, Val MAE: 3322.7242, Val MSE: 45291695.0576, Val R2: 0.5786
Epoch [5/20]
Train Loss: 0.1018, Val Loss: 0.1616
Val RMSE: 6827.1035, Val MAE: 3393.2633, Val MSE: 46609341.9834, Val R2: 0.5664
Epoch [6/20]
Train Loss: 0.0997, Val Loss: 0.1629
Val RMSE: 6847.0272, Val MAE: 3398.9815, Val MSE: 46881781.8115, Val R2: 0.5638
Epoch [7/20]
Train Loss: 0.0977, Val Loss: 0.1737
Val RMSE: 6939.7380, Val MAE: 3465.0464, Val MSE: 48159962.9006, Val R2: 0.5520
Epoch [8/20]
Train Loss: 0.0971, Val Loss: 0.1592
Val RMSE: 6790.5448, Val MAE: 3304.2375,